In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-10 14:05:35.269113: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-10 14:05:35.375048: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-10 14:05:35.375067: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-10 14:05:35.395839: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-10 14:05:35.911733: W tensorflow/stream_executor/platform/de

In [2]:
name_all=[]
auc_roc=[]
name='all_algorithm'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','dmpnn','rf']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100


2022-11-10 14:05:40.163851: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 14:05:40.164355: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-10 14:05:40.164400: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-10 14:05:40.164439: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-10 14:05:40.164476: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

1/1 [==============================] - 0s 393ms/step - loss: 2.4624 - auc: 0.6685
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 2.4246 - auc: 0.8308
Epoch 3/100
1/1 [==============================] - 0s 33ms/step - loss: 2.2639 - auc: 0.8663
Epoch 4/100
1/1 [==============================] - 0s 28ms/step - loss: 2.0131 - auc: 0.8836
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.7978 - auc: 0.9036
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 1.6734 - auc: 0.9192
Epoch 7/100
1/1 [==============================] - 0s 28ms/step - loss: 1.5864 - auc: 0.9309
Epoch 8/100
1/1 [==============================] - 0s 27ms/step - loss: 1.4688 - auc: 0.9578
Epoch 9/100
1/1 [==============================] - 0s 30ms/step - loss: 1.3377 - auc: 0.9807
Epoch 10/100
1/1 [==============================] - 0s 25ms/step - loss: 1.2515 - auc: 0.9881
Epoch 11/100
1/1 [==============================] - 0s 29ms/step - loss: 1.1984 

1/1 [==============================] - 0s 25ms/step - loss: 0.1444 - auc: 0.9979
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1405 - auc: 0.9986
Epoch 90/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1457 - auc: 0.9978
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1468 - auc: 0.9979
Epoch 92/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1487 - auc: 0.9980
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1520 - auc: 0.9971
Epoch 94/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1432 - auc: 0.9979
Epoch 95/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1413 - auc: 0.9981
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1441 - auc: 0.9979
Epoch 97/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1438 - auc: 0.9980
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [3]:
name='remove_rf'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','dmpnn']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 339ms/step - loss: 2.3543 - auc: 0.7954
Epoch 2/100
1/1 [==============================] - 0s 28ms/step - loss: 1.9995 - auc: 0.7667
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 1.8765 - auc: 0.7209
Epoch 4/100
1/1 [==============================] - 0s 28ms/step - loss: 1.7628 - auc: 0.7500
Epoch 5/100
1/1 [==============================] - 0s 33ms/step - loss: 1.5586 - auc: 0.8559
Epoch 6/100
1/1 [==============================] - 0s 33ms/step - loss: 1.4539 - auc: 0.8966
Epoch 7/100
1/1 [==============================] - 0s 29ms/step - loss: 1.3493 - auc: 0.9253
Epoch 8/100
1/1 [==============================] - 0s 28ms/step - loss: 1.2676 - auc: 0.9390
Epoch 9/100
1/1 [==============================] - 0s 30ms/step - loss: 1.1757 - auc: 0.9531
Epoch 10/100
1/1 [==============================] - 0s 29ms/step - loss: 1.0752 - auc: 0.9707
Epoch 11/100
1/1 [==============================] - 0s 33ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.1828 - auc: 0.9961
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1789 - auc: 0.9968
Epoch 90/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1864 - auc: 0.9954
Epoch 91/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1892 - auc: 0.9952
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1862 - auc: 0.9950
Epoch 93/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1801 - auc: 0.9963
Epoch 94/100
1/1 [==============================] - 0s 21ms/step - loss: 0.1777 - auc: 0.9966
Epoch 95/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1834 - auc: 0.9955
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1877 - auc: 0.9951
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1818 - auc: 0.9959
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [4]:
name='remove_dmpnn'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','rf']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 336ms/step - loss: 2.1580 - auc: 0.7819
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.8934 - auc: 0.7918
Epoch 3/100
1/1 [==============================] - 0s 23ms/step - loss: 1.7237 - auc: 0.8385
Epoch 4/100
1/1 [==============================] - 0s 29ms/step - loss: 1.5580 - auc: 0.9005
Epoch 5/100
1/1 [==============================] - 0s 30ms/step - loss: 1.4457 - auc: 0.9327
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 1.3099 - auc: 0.9600
Epoch 7/100
1/1 [==============================] - 0s 28ms/step - loss: 1.2130 - auc: 0.9742
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 1.1341 - auc: 0.9828
Epoch 9/100
1/1 [==============================] - 0s 28ms/step - loss: 1.0605 - auc: 0.9894
Epoch 10/100
1/1 [==============================] - 0s 32ms/step - loss: 1.0101 - auc: 0.9899
Epoch 11/100
1/1 [==============================] - 0s 28ms/step - l

1/1 [==============================] - 0s 28ms/step - loss: 0.1554 - auc: 0.9966
Epoch 89/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1516 - auc: 0.9974
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1486 - auc: 0.9981
Epoch 91/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1486 - auc: 0.9976
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1567 - auc: 0.9975
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1548 - auc: 0.9970
Epoch 94/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1518 - auc: 0.9978
Epoch 95/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1500 - auc: 0.9974
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1459 - auc: 0.9981
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1547 - auc: 0.9967
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [5]:
name='remove_lr'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','dmpnn','rf']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 337ms/step - loss: 2.1030 - auc: 0.6618
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 2.0458 - auc: 0.8303
Epoch 3/100
1/1 [==============================] - 0s 29ms/step - loss: 1.9541 - auc: 0.8572
Epoch 4/100
1/1 [==============================] - 0s 28ms/step - loss: 1.7402 - auc: 0.8780
Epoch 5/100
1/1 [==============================] - 0s 29ms/step - loss: 1.5375 - auc: 0.9000
Epoch 6/100
1/1 [==============================] - 0s 33ms/step - loss: 1.4334 - auc: 0.9094
Epoch 7/100
1/1 [==============================] - 0s 29ms/step - loss: 1.3617 - auc: 0.9189
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 1.3075 - auc: 0.9257
Epoch 9/100
1/1 [==============================] - 0s 30ms/step - loss: 1.1823 - auc: 0.9604
Epoch 10/100
1/1 [==============================] - 0s 31ms/step - loss: 1.0911 - auc: 0.9767
Epoch 11/100
1/1 [==============================] - 0s 35ms/step - l

1/1 [==============================] - 0s 24ms/step - loss: 0.1673 - auc: 0.9959
Epoch 89/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1626 - auc: 0.9970
Epoch 90/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1654 - auc: 0.9966
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1593 - auc: 0.9978
Epoch 92/100
1/1 [==============================] - 0s 22ms/step - loss: 0.1658 - auc: 0.9967
Epoch 93/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1656 - auc: 0.9971
Epoch 94/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1611 - auc: 0.9970
Epoch 95/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1622 - auc: 0.9969
Epoch 96/100
1/1 [==============================] - 0s 31ms/step - loss: 0.1677 - auc: 0.9962
Epoch 97/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1658 - auc: 0.9968
Epoch 98/100
1/1 [==============================] - 0s 28ms/step - loss: 

In [6]:
name='remove_fcnn'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','lr','dmpnn','rf']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 336ms/step - loss: 2.1023 - auc: 0.7427
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 1.8384 - auc: 0.8102
Epoch 3/100
1/1 [==============================] - 0s 24ms/step - loss: 1.6584 - auc: 0.8920
Epoch 4/100
1/1 [==============================] - 0s 29ms/step - loss: 1.4761 - auc: 0.9333
Epoch 5/100
1/1 [==============================] - 0s 34ms/step - loss: 1.3554 - auc: 0.9571
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 1.2443 - auc: 0.9773
Epoch 7/100
1/1 [==============================] - 0s 25ms/step - loss: 1.1664 - auc: 0.9846
Epoch 8/100
1/1 [==============================] - 0s 26ms/step - loss: 1.0950 - auc: 0.9893
Epoch 9/100
1/1 [==============================] - 0s 28ms/step - loss: 1.0321 - auc: 0.9918
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 0.9666 - auc: 0.9951
Epoch 11/100
1/1 [==============================] - 0s 31ms/step - l

1/1 [==============================] - 0s 29ms/step - loss: 0.1324 - auc: 0.9983
Epoch 89/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1446 - auc: 0.9972
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1386 - auc: 0.9982
Epoch 91/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1393 - auc: 0.9972
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1424 - auc: 0.9971
Epoch 93/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1374 - auc: 0.9978
Epoch 94/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1369 - auc: 0.9982
Epoch 95/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1367 - auc: 0.9979
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1387 - auc: 0.9977
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1356 - auc: 0.9981
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [7]:
name='remove_xgb'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['fcnn','lr','dmpnn','rf']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 338ms/step - loss: 2.3943 - auc: 0.4878
Epoch 2/100
1/1 [==============================] - 0s 24ms/step - loss: 2.0102 - auc: 0.8184
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 2.0771 - auc: 0.8446
Epoch 4/100
1/1 [==============================] - 0s 35ms/step - loss: 1.9964 - auc: 0.8658
Epoch 5/100
1/1 [==============================] - 0s 30ms/step - loss: 1.8419 - auc: 0.8808
Epoch 6/100
1/1 [==============================] - 0s 28ms/step - loss: 1.6455 - auc: 0.8939
Epoch 7/100
1/1 [==============================] - 0s 33ms/step - loss: 1.4612 - auc: 0.9159
Epoch 8/100
1/1 [==============================] - 0s 28ms/step - loss: 1.3770 - auc: 0.9213
Epoch 9/100
1/1 [==============================] - 0s 25ms/step - loss: 1.3396 - auc: 0.9171
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 1.2896 - auc: 0.9216
Epoch 11/100
1/1 [==============================] - 0s 29ms/step - l

1/1 [==============================] - 0s 23ms/step - loss: 0.1830 - auc: 0.9959
Epoch 89/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1716 - auc: 0.9976
Epoch 90/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1735 - auc: 0.9969
Epoch 91/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1765 - auc: 0.9966
Epoch 92/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1784 - auc: 0.9960
Epoch 93/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1774 - auc: 0.9962
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1719 - auc: 0.9973
Epoch 95/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1727 - auc: 0.9972
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1875 - auc: 0.9942
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1736 - auc: 0.9968
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [8]:
de=pd.DataFrame()
de['name']=name_all
de['auc_roc']=auc_roc
de.to_csv('5model_remove_one_model.csv',index=False)
print(de)

            name   auc_roc
0  all_algorithm  0.940907
1      remove_rf  0.944272
2   remove_dmpnn  0.939112
3      remove_lr  0.931265
4    remove_fcnn  0.941136
5     remove_xgb  0.939297
